**IMDB sentiment analysis using DistillBERT**

In [ ]:
!pip install transformers

In [ ]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

In [ ]:
# 텍스트 형태의 데이터를 data, label 각각을 관리하기 위한 리스트 형태로 변환
# train data, label & test data, label의 25,000개 데이터로 나뉨
from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels

#구현 필요
train_texts, train_labels = read_imdb_split('acIImdb/train')
test_texts, test_labels = read_imdb_split('acIImdb/test')

In [ ]:
# 문장 토큰화를 위해 tokenizer로드
# DistillBERT를 학습시킬 때 사용한 tokenizer를 활용해야 pre-trained LM을 로드한 후 학습할 때 원하는 대로 임베딩 값이 입력됨
# WordPiece라는 토큰화 및 임베딩 방식을 활용함 (단어를 더 쪼개서 sub-word 토큰을 만듦)
# 워드 임베딩에 비해 적은 수의 토큰으로 ㅁ낳은 단어를 포현할 수 있고, 처음 보는 단어도 토큰을 조합해서 임베딩을 나타낼 수 있게 됨
from transformers import DistilBertTokenizerFast
#구현 필요
tokenizer= DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
# 로딩된 tokenizer 활용하여 토큰화 진행
# truncation 및 padding은 max_length 길이에 맞지 않을 때 자르거나 padding을 붙이는 것에 대한 파라미터를 의미함
#구현 필요
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [1]:
# 모델 입력을 위한 Pytorch 데이터셋 생성
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

#구현 필요
train_dataset =  IMDbDataset(test_encodings, train_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [ ]:
# 학습을 위한 각종 하이퍼파라미터 설정 및 pre-trained LM 로딩

from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=25,  # batch size per device during training
    per_device_eval_batch_size=50,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=250,
)

#구현 필요
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

# Transformers 라이브러리의 Trainer 객체를 이용해 편리하게 학습 진행 가능
def compute_metrics(pred):    # 기본적으로 Trainer에서 evaluate 메서드를 호출하면 loss 및 예측 속도 측면에서의 성능만 기록됨
    labels = pred.label_ids     # 이에 추가적인 성능 지표를 사용하고 싶다면 좌측과 같은 형식을 갖춘 함수를 정의 후 trainer 객체 생성 시 인자로 넣어주어야 함
    preds = pred.predictions.argmax(-1)
    
    acc = accuracy_score(labels, preds)    
    return {
        'accuracy': acc
    }

#구현 필요
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics # 없으면 정확도 출력 x only loss만 출력
)

# 테스트 결과 93% 정도의 정확도를 기록함
# 지난 주 실습보다 훨씬 적은 개수의 학습 데이터를 사용하였지만 pre_trained LM의 효과로 훨씬 높은 정확도를 달성

In [ ]:
# 테스트 함수
metrics = trainer.evaluate(eval_dataset=test_dataset)
#구현 필요

In [ ]:
metrics

**Sentiment analysis**

**transformer에서는 pipeline 이라는 high-level API를 통해 손쉽게 이미 학습되어있는 모델에 기반한 다양한 NLP task를 수행할 수 있음**
**Pipeline에서는 로드하여 사용하는 모델이 있으나 객체 내부적으로 알아서 진행하는 것이며 직접 모델을 PyTorch로 불러와서 활용하는 방법도 가능함**


In [ ]:
# pipeline을 호출할 대 현재 목표로 하는 task를 입력하면 관련 모델을 로드하여 곧바로 활용할 수 있도록 객체를 만들어 줌
from transformers import pipeline

#구현 필요
nlp = pipeline("sentiment-analysis")

#구현 필요
result = nlp("I hate you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

#구현 필요
result = nlp("I Love you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

**Question answering**

**Question answering이란 지문과 문제를 풀고 단답형 주관식에 대한 답을 내는 task를 말함**

In [ ]:
# pipeline 생성 시 question-answering이라는 task를 입력하면 되고, question과 context 정보를 주면 답을 반환해 줌
from transformers import pipeline

#구현 필요
nlp = pipeline("question-answering")

context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/question-answering/run_squad.py script.
"""

In [ ]:
#구현 필요
result = nlp(question="what is extractive question answering?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

#구현 필요
result = nlp(question="what is a good example of a question answering dataset?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

**Text generation**

In [ ]:
# pipeline 생성 시 text-generation이라는 task를 입력하면 문장 생성 task를 수행가능함
# 첫 단어 몇개를 나열해주면 그 뒤를 알아서 생성
# max_length: 생성할 문장의 최대 단어 수
# do_sample: 매 단어 생성시 top k개 중 샘플링 할 것인지에 대한 매개변수를 의미함

from transformers import pipeline

#구현 필요
text_generator = pipeline("text-generation")
print(text_generator("As far as I am concerned, I will", max_length=50, do_sample=False))

In [ ]:
# do_sample을 설정하고 몇개의 상위 확률 단어들 중에 고를 것인지 top_k로 명시해주면 다른 생성 결과를 볼 수 있다

#구현 필요
print(text_generator("As far as I am concerned, I will", max_length=50, do_sample=True, top_k=3))

**Using the specific model**

In [ ]:
# text generation pipeline을 해당 모델을 활용해서 만들 수 있음
# 모든 모델이 pipeline 기능이 지원되는 것은 아니며, pipeline 지원이 안되는 모델은 해당 모델 github 페이지 등을 참조하여 모델을 직접 활용하여야
#구현 필요
text_generator = pipeline("text-generation", model="skt/kogpt2-base-v2")
print(text_generator("요즘 부쩍 날씨가 추워졌습니다", max_length=50, do_sample =False))